# Introduction to the Survival Analysis

This is a short introduction to how to use statistical methods with survival analysis with TCGA breast cancer data.


## Methods


> **See Also**: For a more in-depth tour about statistical methods you can visit the following links:


## Data

- For this tutorial we will use two techniques to divide data into training and validation with TCGA data:
    - Bootstrap resampling
    - Cross validation
    -- k-folds
    -- Stratified k-folds
- The data we are going to use is right censored which means not all patients deceased

> For **python** libraries:
> - lifelines
> -  sklearn


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import numpy as np
from sklearn.model_selection import StratifiedKFold

Load data from CSV file with 1083 breast cancer patients from TCGA

In [ ]:
df = pd.read_csv('Dataset/BRCA_TCGA_Tutorial.csv')
df.head()
print(len(df))

In [ ]:
# Look at the categorical and numerical variables
summary_categorical = []
for column in df.columns:
  if df[column].dtype == object:
    summary_categorical.append(column)
    print(df[column].value_counts())
    print(f"----------------------------------")

summary_numerical = []
for column in df.columns:
  if df[column].dtype != object:
    print(f"{column} : Minimum: {min((df[column].unique()))}, Maximum: {max((df[column].unique()))} ")
    summary_numerical.append(column)
    print(f"----------------------------------")

In [ ]:
# Convert all string categorical variables into integer labels
encoder = LabelEncoder()
for column in df.columns:
  if df[column].dtype == object:
    df[column] = encoder.fit_transform(df[column])

# Kaplan-Meier Estimator

In [ ]:
# here to specify which event we will make our study about OS or PFI etc.
event = 'OS'
time = 'OS Time'
# remove the rows with NaNs
print(len(df))
df_surv = df.dropna(subset=[event, time])
print(len(df_surv))

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(durations=df_surv[time], event_observed=df_surv[event])

kmf.survival_function_.plot(figsize=(8,5))
plt.title('Survival Curve estimated with Kaplan-Meier Fitter')
plt.show()

kmf.plot_survival_function(figsize=(8,5))
plt.title('Survival Curve estimated with Kaplan-Meier Fitter with confidence intervals')
plt.show()


In [ ]:
# here is to show the KM estimated survival scores based on OS
print(kmf.survival_function_)

## Survival Function of Different Groups

In [ ]:
print(df_surv['Immune Subtype'].value_counts())
df_surv['Immune Subtype'].hist()

In [ ]:
Patient_C1 = (df_surv['Immune Subtype'] == 0)
Patient_C2 = (df_surv['Immune Subtype'] == 1)
Patient_C3 = (df_surv['Immune Subtype'] == 2)
Patient_C4 = (df_surv['Immune Subtype'] == 3)
Patient_C6 = (df_surv['Immune Subtype'] == 4)

ax = plt.subplot()
kmf = KaplanMeierFitter()

kmf.fit(durations=df_surv[Patient_C1][time],
        event_observed=df_surv[Patient_C1][event],
        label='C1')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_C2][time],
        event_observed=df_surv[Patient_C2][event],
        label='C2')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_C3][time],
        event_observed=df_surv[Patient_C3][event],
        label='C3')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_C4][time],
        event_observed=df_surv[Patient_C4][event],
        label='C4')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_C6][time],
        event_observed=df_surv[Patient_C6][event],
        label='C6')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)
plt.title('Survival Function based on Immune Subtype')
plt.show()


In [ ]:
df['TCGA Subtype'].hist()

In [ ]:
Patient_BRCA_LumA = (df_surv['TCGA Subtype'] == 2)
Patient_BRCA_LumB = (df_surv['TCGA Subtype'] == 3)
Patient_BRCA_Basal = (df_surv['TCGA Subtype'] == 0)
Patient_BRCA_Normal = (df_surv['TCGA Subtype'] == 4)
Patient_Her2 = (df_surv['TCGA Subtype'] == 1)


ax = plt.subplot()
kmf = KaplanMeierFitter()

kmf.fit(durations=df_surv[Patient_BRCA_LumA][time],
        event_observed=df_surv[Patient_BRCA_LumA][event],
        label='BRCA LumA')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_BRCA_LumB][time],
        event_observed=df_surv[Patient_BRCA_LumB][event],
        label='BRCA LumB')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_BRCA_Basal][time],
        event_observed=df_surv[Patient_BRCA_Basal][event],
        label='BRCA Basal')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_BRCA_Normal][time],
        event_observed=df_surv[Patient_BRCA_Normal][event],
        label='BRCA Normal')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_surv[Patient_Her2][time],
        event_observed=df_surv[Patient_Her2][event],
        label='Her2')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)
plt.title('Survival Function based on TCGA Subtype')
plt.show()


In [ ]:


# to do the survival analysis with continuous variable
# select a threshold based on different methods (mean, median, mode etc)
feature = 'TIL Regional Fraction'
df_variable = df.copy()

print(len(df_variable))
df_variable = df_variable.dropna(subset=[feature,time,event])
threshold = df_variable[feature].median()
print(len(df_variable))

# to specify the time maximum range
df_variable = df_variable[df_variable[time]<6000]

print("The threshold value: {:.9f}".format(threshold))

Patient_group_1 = (df_variable[feature] >= threshold)
Patient_group_2 = (df_variable[feature] < threshold)

ax = plt.subplot()
kmf = KaplanMeierFitter()


kmf.fit(durations=df_variable[Patient_group_1][time],
        event_observed=df_variable[Patient_group_1][event],
        label=feature+' >= Threshold')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

kmf.fit(durations=df_variable[Patient_group_2][time],
        event_observed=df_variable[Patient_group_2][event],
        label=feature+' < Threshold')
kmf.survival_function_.plot(figsize=(8,5), ax=ax)

plt.title('Survival Function based on '+ feature+' median')
plt.show()


## Log-rank Test

In [ ]:
results = logrank_test(df_surv[Patient_BRCA_LumA][time], df_surv[Patient_Her2][time],
                       event_observed_A=df_surv[Patient_BRCA_LumA][event], event_observed_B=df_surv[Patient_Her2][event])

print("p-value:{:.9f}".format(results.p_value))
print("test statistic:{:.2f}".format(results.test_statistic))


In [ ]:
results = logrank_test(df_surv[Patient_BRCA_Basal][time], df_surv[Patient_BRCA_Normal][time],
                       event_observed_A=df_surv[Patient_BRCA_Basal][event], event_observed_B=df_surv[Patient_BRCA_Normal][event])

print("p-value:{:.9f}".format(results.p_value))
print("test statistic:{:.2f}".format(results.test_statistic))

## Cox Proportional-Hazards Model
A negative coefficient means that an increase in the value of the corresponding covariate is associated with a decreased hazard rate, implying a lower risk of experiencing the event. The Concordance Index evaluates the accuracy of the ordering of predicted time with random predictions <= 0.5 and perfect concordance of 1.0.


In [ ]:
# Prepare the dataframe for fitting
columns_selected = ['Immune Subtype',
                    'TCGA Subtype',
                    'age',
                    'stage',
                    'Oncotree Code',
                    'tumour_stage',
                    'TIL Regional Fraction', time, event]
df_cox = df.copy()
df_cox = df_cox[columns_selected]
print(len(df_cox))
#clean the data from NaN values
df_cox = df_cox.dropna(subset=columns_selected)
print(len(df_cox))


In [ ]:
#initialize a fitter
coxph = CoxPHFitter()

# Fit Cox Proportional Hazards model on the selected data
coxph.fit(df_cox,
          duration_col=time,
          event_col=event)

In [ ]:
# Print Cox Proportional Hazards model Summary
# coxph.print_summary()
# Print Cox Proportional Hazards model coefficient for each covariate
print(pd.DataFrame({'Covariate': df_cox.drop([time, event], axis=1).columns, 'Coefficient': coxph.summary['coef'].values}))
print("Concordance:{:.2f}".format(coxph.concordance_index_ ))

In [ ]:
coxph.plot()

# 1. Bootstrap Resampling

In [ ]:
n_bootstrap = 10
coefficients = []
concordances = []

# Loop through bootstrap runs
for i in range(n_bootstrap):
    # Bootstrap sampling with replacement
    bootstrap_sample = df_cox.sample(frac=1.0, replace=True)

    # Fit Cox Proportional Hazards model
    cph = CoxPHFitter()
    cph.fit(bootstrap_sample, duration_col=time, event_col=event)

    # Store coefficients from the current run
    coefficients.append(cph.summary['coef'].values)
    concordances.append(cph.concordance_index_)

# Calculate the average coefficients
average_coefficients = np.mean(coefficients, axis=0)
average_concordance = np.mean(concordances, axis=0)

# Display the average coefficients
print("Average Coefficients:")
print(pd.DataFrame({'Covariate': df_cox.drop([time, event], axis=1).columns, 'Coefficient': average_coefficients}))
print("Average concordances:")
print(average_concordance)
print("Standard deviation of concordances:")
print(np.std(average_concordance))

# Cross-validation (Stratified k-folds)

In [ ]:
cph_scores = []
cph = CoxPHFitter()
skf = StratifiedKFold(n_splits=10, shuffle=True)

#loop over 10 folds which stratified based on the event
for fold, (train_idx, test_idx) in enumerate(skf.split(df_cox, df_cox[event])):
        #locate the indices of train and test in the original dataset with the selected covariances
        train_set = df_cox.iloc[train_idx, :]
        test_set = df_cox.iloc[test_idx, :]

        # Fit Cox Proportional Hazards model
        cph.fit(train_set, duration_col=time, event_col=event)

        #do scoring with concordance index as a return value on test data using the trained model
        cph_scores.append(cph.score(test_set, scoring_method='concordance_index'))

print(np.mean(cph_scores), np.std(cph_scores))
